In [142]:
import pandas as pd
import numpy as np
import random
import glob

In [143]:
# Charger les fichiers un par un
df_routes = pd.read_csv(r'C:\Users\Durant.Nguefack\Desktop\projet_transport\archive\routes.csv', sep=';')
df_routes_coords = pd.read_csv(r'C:\Users\Durant.Nguefack\Desktop\projet_transport\archive\routes_with_coordinates.csv', sep=';')
df_stops = pd.read_csv(r'C:\Users\Durant.Nguefack\Desktop\projet_transport\archive\stops.csv', sep=';')
df_stop_times = pd.read_csv(r'C:\Users\Durant.Nguefack\Desktop\projet_transport\archive\stop_times.csv', sep=';')
df_trips = pd.read_csv(r'C:\Users\Durant.Nguefack\Desktop\projet_transport\archive\trips.csv', sep=';')


In [144]:
df_routes = df_routes[['route_id', 'route_long_name']]


In [145]:
df_stops = df_stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']]


In [146]:
df_stop_times = df_stop_times[['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence']]


In [147]:
df_trips = df_trips[['trip_id', 'route_id']]


In [148]:
# Étape 1 : stop_times + trips (par trip_id)
df_copy = pd.merge(df_stop_times, df_trips, on='trip_id', how='left')

# Étape 2 : ajouter les infos de route
df_copy = pd.merge(df_copy, df_routes, on='route_id', how='left')

# Étape 3 : ajouter les infos d’arrêt
df_copy = pd.merge(df_copy, df_stops, on='stop_id', how='left')


In [149]:
# Supprimer les lignes avec données critiques manquantes
df_copy.dropna(subset=['trip_id', 'arrival_time', 'departure_time', 'stop_sequence'], inplace=True)

print("Forme finale du DataFrame prêt :", df_copy.shape)
print(df_copy.head())


Forme finale du DataFrame prêt : (15017, 10)
    trip_id arrival_time departure_time  stop_id  stop_sequence route_id  \
0  100E-1-1     08:05:00       08:05:00      637              1     100E   
1  100E-1-2     09:35:00       09:35:00      637              1     100E   
2  100E-1-3     11:25:00       11:25:00      637              1     100E   
3  100E-1-4     12:50:00       12:50:00      637              1     100E   
4  100E-1-5     15:45:00       15:45:00      637              1     100E   

                route_long_name            stop_name   stop_lat   stop_lon  
0  Sadhashivanagar - BTM Layout  BTM Layout Bus Stop  12.916363  77.615991  
1  Sadhashivanagar - BTM Layout  BTM Layout Bus Stop  12.916363  77.615991  
2  Sadhashivanagar - BTM Layout  BTM Layout Bus Stop  12.916363  77.615991  
3  Sadhashivanagar - BTM Layout  BTM Layout Bus Stop  12.916363  77.615991  
4  Sadhashivanagar - BTM Layout  BTM Layout Bus Stop  12.916363  77.615991  


In [151]:
df_copy['stop_sequence'].value_counts().sort_index()


stop_sequence
0       15
1    14987
2       15
Name: count, dtype: int64

In [152]:
df_copy['arrival_time'] = pd.to_datetime(df_copy['arrival_time'], format='%H:%M:%S', errors='coerce')
df_copy['departure_time'] = pd.to_datetime(df_copy['departure_time'], format='%H:%M:%S', errors='coerce')


In [153]:
df_copy['duree_arret_min'] = (df_copy['departure_time'] - df_copy['arrival_time']).dt.total_seconds() / 60


In [156]:
# Durée totale du trajet
df_durees = df_copy.groupby('trip_id')['duree_arret_min'].sum().reset_index()
df_durees.rename(columns={'duree_arret_min': 'duree_totale_min'}, inplace=True)

# Nombre d’arrêts
df_arrets = df_copy.groupby('trip_id')['stop_sequence'].count().reset_index()
df_arrets.rename(columns={'stop_sequence': 'nombre_arrets'}, inplace=True)

# Fusionner les deux
df_analyse = pd.merge(df_durees, df_arrets, on='trip_id')


In [157]:
df_analyse['panne_probable'] = ((df_analyse['duree_totale_min'] > 60) | 
                                (df_analyse['nombre_arrets'] > 25)).astype(int)


In [158]:
print(df_analyse.head())
print(df_analyse['panne_probable'].value_counts())


    trip_id  duree_totale_min  nombre_arrets  panne_probable
0  100E-1-1               0.0              1               0
1  100E-1-2               0.0              1               0
2  100E-1-3               0.0              1               0
3  100E-1-4               0.0              1               0
4  100E-1-5               0.0              1               0
panne_probable
0    15017
Name: count, dtype: int64


In [159]:
# Repartir du vrai fichier stop_times seul
df_stop_times = pd.read_csv(r'C:\Users\Durant.Nguefack\Desktop\projet_transport\archive\stop_times.csv', sep=';')

# Vérifier combien de lignes il y a par trip_id
df_stop_times['trip_id'].value_counts().head(10)


trip_id
MP-60A-1-14502    1
100E-1-1          1
100E-1-2          1
100E-1-3          1
100E-1-4          1
100E-1-5          1
100E-1-6          1
100E-1-7          1
104-1-8           1
MP-60A-1-14531    1
Name: count, dtype: int64